In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import time
from datetime import timedelta

# Data

In [ ]:
train_df = pd.read_json('train.json',  lines = True).drop(['index'], axis = 1)
test_df = pd.read_json('test.json', lines = True).drop(['index'], axis = 1)

def cut_short(seq, maxlen = 107):
    return seq[:maxlen]

    
test_df[['sequence']]=test_df['sequence'].apply(cut_short)
test_df[['structure']]=test_df['structure'].apply(cut_short)
test_df[['predicted_loop_type']]=test_df['predicted_loop_type'].apply(cut_short)


In [3]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSXZ')}

target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    return np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )

train_inputs = preprocess_inputs(train_df.loc[train_df.SN_filter == 1])
train_labels = np.array(train_df.loc[train_df.SN_filter == 1][target_cols].values.tolist()).transpose((0, 2, 1))

In [4]:
from sklearn.model_selection import train_test_split

seed = 123
X_train, X_test, Y_train, Y_test = train_test_split(train_inputs, train_labels, test_size = 0.1, random_state = seed)

print(np.shape(X_train)) #1430 samples de séquences de 107 de long et de 3=sequence,structure, predicted loop type

(1430, 107, 3)


# Model

In [6]:
import tensorflow.keras.layers as L
def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True))


def build_model(seq_len=107, pred_len=68, dropout=0.5, embed_dim=75, hidden_dim=128):
    inputs = L.Input(shape=(seq_len, 3))

    embed = L.Embedding(input_dim=len(token2int), output_dim=embed_dim)(inputs)
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))

    hidden = gru_layer(hidden_dim, dropout)(reshaped)
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    
    # Since we are only making predictions on the first part of each sequence, we have
    # to truncate it
    truncated = hidden[:, :pred_len]
    
    out = L.Dense(5, activation='linear')(truncated)

    model = tf.keras.Model(inputs=inputs, outputs=out)

    model.compile(tf.keras.optimizers.Adam(), loss='mse',  metrics=['accuracy'])
    
    return model

In [25]:
def prepare_window_training(base_index, n_bases,X_train, Y_train, X_test, Y_test):
   
    start_index=int(i-(window_size-1)/2)
    stop_index=int(i+(window_size-1)/2 +1) #not included
    full_array=[]
    for sample_index in range(len(X_train)):
        sample_array=X_train[sample_index]

        if start_index<0:
            out_array=np.array([[14]*len(sample_array[0])]*(-int(start_index)))
            studied_cut_array=sample_array[0:stop_index]
            studied_array=np.concatenate((out_array,studied_cut_array))

        elif stop_index>68:
            out_array=np.array([[14]*len(sample_array[0])]*(int(stop_index-68)))
            studied_cut_array=sample_array[start_index:68]
            studied_array=np.concatenate((studied_cut_array,out_array))

        else :
            studied_array=sample_array[start_index:stop_index]
        full_array.append(studied_array)
    
    full_array_test=[]
    for sample_index in range(len(X_test)):
        sample_array=X_test[sample_index]

        if start_index<0:
            out_array=np.array([[14]*len(sample_array[0])]*(-int(start_index)))
            studied_cut_array=sample_array[0:stop_index]
            studied_array=np.concatenate((out_array,studied_cut_array))

        elif stop_index>68:
            out_array=np.array([[14]*len(sample_array[0])]*(int(stop_index-68)))
            studied_cut_array=sample_array[start_index:68]
            studied_array=np.concatenate((studied_cut_array,out_array))

        else :
            studied_array=sample_array[start_index:stop_index]
        full_array_test.append(studied_array)

    X_train_window=np.array(full_array)
    Y_train_window=Y_train[:,i,:]
    X_test_window=np.array(full_array_test)
    Y_test_window=Y_test[:,i,:]
    
    return X_train_window, Y_train_window, X_test_window, Y_test_window

# Window mode


In [135]:
def prepare_window_prediction(base_index, n_bases, X_to_predict):
   
    start_index=int(i-(window_size-1)/2)
    stop_index=int(i+(window_size-1)/2 +1) #not included
    full_array=[]
    for sample_index in range(len(X_to_predict)):
        sample_array=X_to_predict[sample_index]

        if start_index<0:
            out_array=np.array([[14]*len(sample_array[0])]*(-int(start_index)))
            studied_cut_array=sample_array[0:stop_index]
            studied_array=np.concatenate((out_array,studied_cut_array))

        elif stop_index>68:
            out_array=np.array([[14]*len(sample_array[0])]*(int(stop_index-68)))
            studied_cut_array=sample_array[start_index:68]
            studied_array=np.concatenate((studied_cut_array,out_array))

        else :
            studied_array=sample_array[start_index:stop_index]
        full_array.append(studied_array)
        
    return np.array(full_array)

In [ ]:
#mode fenêtré:
window_size=21 #doit être un nombre impair
X_train=X_train[:,:68,:]
X_test=X_test[:,:68,:]

X_to_predict = preprocess_inputs(test_df)[:,:68,:]
Y_predicted = np.empty(shape=(len(X_to_predict), 68, 5))
print(len(X_to_predict))


start_time = time.monotonic()

for i in range(68): #on parcourt toute la séquence
    
    print('Training for base', i, '===================================')

    X_train_window, Y_train_window, X_test_window, Y_test_window = prepare_window_training(i, 68, 
                                                                                   X_train, Y_train, 
                                                                                   X_test, Y_test)
    #ici il faut faire le fit sur le full_array de dimensions 2160*21*14=nombredeséquences*window_size*nombredeparamètresd'input
    #ensuite on lance le modèle sur les vraies données, 
    #et on stocke le résultat [a,b,c] dans l'item i une liste du type [[1, 0, 0], [0, 1, 0.3], [0.4, 0.7, 1], ...], de dimensions 68*3
    
    model = build_model(seq_len=21, embed_dim=len(token2int))
    model.summary()
    Y_train_window=np.array([np.tile(Y_train_window[i],(21,1)) for i in range(len(Y_train_window))])
    Y_test_window=np.array([np.tile(Y_test_window[i],(21,1)) for i in range(len(Y_test_window))])

    print(np.shape(X_train_window))
    print(np.shape(Y_train_window))
    print(np.shape(X_test_window))
    print(np.shape(Y_test_window))
    
    history = model.fit(
    X_train_window, 
    Y_train_window,
    validation_data=(X_test_window, Y_test_window),
    batch_size=32,
    epochs=30,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=5),
        tf.keras.callbacks.ModelCheckpoint('model.h5')
    ]
    )
    
    plt.plot(history.history['val_loss'])
    plt.plot(history.history['loss'])
    plt.legend(['val_loss {0}'.format(i), 'loss {0}'.format(i)], loc='upper right')

    
    print('Evaluation for base', i, '===================================')

# la prédiction qui suit ne marche pas encore 
    X_predic=prepare_window_prediction(i,68,X_to_predict)
    
    for seq in range(len(X_predic)) :
        if seq%100==0:
            print(seq)
        base_predict=model.predict(X_predic[seq])
        mean_predict=np.mean(base_predict[0], axis=0)
        Y_predicted[seq, i, :]=mean_predict
#         print(mean_predict)

end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))

3634
Training for base 0 ===================================
Model: "functional_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(None, 21, 3)]           0         
_________________________________________________________________
embedding_37 (Embedding)     (None, 21, 3, 15)         225       
_________________________________________________________________
tf_op_layer_Reshape_37 (Tens [(None, 21, 45)]          0         
_________________________________________________________________
bidirectional_111 (Bidirecti (None, 21, 256)           134400    
_________________________________________________________________
bidirectional_112 (Bidirecti (None, 21, 256)           296448    
_________________________________________________________________
bidirectional_113 (Bidirecti (None, 21, 256)           296448    
__________________________________________________________

Epoch 19/30
45/45 - 5s - loss: 0.5576 - accuracy: 0.8047 - val_loss: 0.5797 - val_accuracy: 0.7987
Epoch 20/30
45/45 - 5s - loss: 0.5364 - accuracy: 0.8042 - val_loss: 0.5559 - val_accuracy: 0.7954
Epoch 21/30
45/45 - 6s - loss: 0.5150 - accuracy: 0.8029 - val_loss: 0.5313 - val_accuracy: 0.7969
Epoch 22/30
45/45 - 6s - loss: 0.5130 - accuracy: 0.8041 - val_loss: 0.5481 - val_accuracy: 0.7987
Epoch 23/30
45/45 - 6s - loss: 0.5102 - accuracy: 0.8009 - val_loss: 0.5242 - val_accuracy: 0.8167
Epoch 24/30
45/45 - 5s - loss: 0.4978 - accuracy: 0.8036 - val_loss: 0.5274 - val_accuracy: 0.8230
Epoch 25/30
45/45 - 5s - loss: 0.4824 - accuracy: 0.8062 - val_loss: 0.5008 - val_accuracy: 0.7987
Epoch 26/30
45/45 - 5s - loss: 0.4745 - accuracy: 0.8038 - val_loss: 0.4908 - val_accuracy: 0.8095
Epoch 27/30
45/45 - 5s - loss: 0.4757 - accuracy: 0.8084 - val_loss: 0.5319 - val_accuracy: 0.8059
Epoch 28/30
45/45 - 5s - loss: 0.4598 - accuracy: 0.8079 - val_loss: 0.4866 - val_accuracy: 0.8269
Epoch 29/3

<ipython-input-149-8c292374b454>:42: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.subplot()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Training for base 2 ===================================
Model: "functional_79"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, 21, 3)]           0         
_________________________________________________________________
embedding_39 (Embedding)     (None, 21, 3, 15)         225       
_________________________________________________________________
tf_op_layer_Reshape_39 (Tens [(None, 21, 45)]          0         
_________________________________________________________________
bidirectional_117 (Bidirecti (None, 21, 256)           134400    
_________________________________________________________________
bidirectional_118 (Bidirecti (None, 21, 256)           296448    
________________________

45/45 - 5s - loss: 0.1473 - accuracy: 0.3360 - val_loss: 0.1289 - val_accuracy: 0.2977
Epoch 18/30
45/45 - 5s - loss: 0.1492 - accuracy: 0.3398 - val_loss: 0.1298 - val_accuracy: 0.3624
Epoch 19/30
45/45 - 4s - loss: 0.1477 - accuracy: 0.3475 - val_loss: 0.1314 - val_accuracy: 0.3046
Epoch 20/30
45/45 - 6s - loss: 0.1472 - accuracy: 0.3511 - val_loss: 0.1281 - val_accuracy: 0.3522
Epoch 21/30
45/45 - 7s - loss: 0.1436 - accuracy: 0.3781 - val_loss: 0.1260 - val_accuracy: 0.3573
Epoch 22/30
45/45 - 5s - loss: 0.1428 - accuracy: 0.3721 - val_loss: 0.1258 - val_accuracy: 0.3792
Epoch 23/30
45/45 - 7s - loss: 0.1424 - accuracy: 0.3727 - val_loss: 0.1257 - val_accuracy: 0.3789
Epoch 24/30
45/45 - 6s - loss: 0.1413 - accuracy: 0.3727 - val_loss: 0.1277 - val_accuracy: 0.3645
Epoch 25/30
45/45 - 6s - loss: 0.1427 - accuracy: 0.3710 - val_loss: 0.1259 - val_accuracy: 0.3648
Epoch 26/30
45/45 - 5s - loss: 0.1415 - accuracy: 0.3850 - val_loss: 0.1253 - val_accuracy: 0.3738
Epoch 27/30
45/45 - 5s

...

Epoch 2/30
45/45 - 4s - loss: 0.1675 - accuracy: 0.2777 - val_loss: 0.1314 - val_accuracy: 0.2845
Epoch 3/30
45/45 - 4s - loss: 0.1611 - accuracy: 0.2666 - val_loss: 0.1294 - val_accuracy: 0.2893
Epoch 4/30
45/45 - 4s - loss: 0.1568 - accuracy: 0.2846 - val_loss: 0.1237 - val_accuracy: 0.3013
Epoch 5/30
45/45 - 4s - loss: 0.1588 - accuracy: 0.2858 - val_loss: 0.1370 - val_accuracy: 0.2656
Epoch 6/30
45/45 - 4s - loss: 0.1548 - accuracy: 0.2862 - val_loss: 0.1180 - val_accuracy: 0.2809
Epoch 7/30
45/45 - 4s - loss: 0.1530 - accuracy: 0.2847 - val_loss: 0.1257 - val_accuracy: 0.2917
Epoch 8/30
45/45 - 4s - loss: 0.1536 - accuracy: 0.2903 - val_loss: 0.1367 - val_accuracy: 0.2833
Epoch 9/30
45/45 - 5s - loss: 0.1464 - accuracy: 0.2916 - val_loss: 0.1120 - val_accuracy: 0.2710
Epoch 10/30
45/45 - 4s - loss: 0.1459 - accuracy: 0.2833 - val_loss: 0.1245 - val_accuracy: 0.2923
Epoch 11/30
45/45 - 4s - loss: 0.1421 - accuracy: 0.2901 - val_loss: 0.1185 - val_accuracy: 0.2920
Epoch 12/30
45/45 

3400
3500
3600
Training for base 12 ===================================
Model: "functional_99"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_50 (InputLayer)        [(None, 21, 3)]           0         
_________________________________________________________________
embedding_49 (Embedding)     (None, 21, 3, 15)         225       
_________________________________________________________________
tf_op_layer_Reshape_49 (Tens [(None, 21, 45)]          0         
_________________________________________________________________
bidirectional_147 (Bidirecti (None, 21, 256)           134400    
_________________________________________________________________
bidirectional_148 (Bidirecti (None, 21, 256)           296448    
_________________________________________________________________
bidirectional_149 (Bidirecti (None, 21, 256)           296448    
_______________________________________________

Epoch 19/30
45/45 - 4s - loss: 0.1296 - accuracy: 0.3751 - val_loss: 0.2086 - val_accuracy: 0.3887
Epoch 20/30
45/45 - 4s - loss: 0.1298 - accuracy: 0.3815 - val_loss: 0.2281 - val_accuracy: 0.3890
Epoch 21/30
45/45 - 5s - loss: 0.1310 - accuracy: 0.3914 - val_loss: 0.2044 - val_accuracy: 0.3923
Epoch 22/30
45/45 - 4s - loss: 0.1219 - accuracy: 0.3787 - val_loss: 0.2055 - val_accuracy: 0.3795
Epoch 23/30
45/45 - 4s - loss: 0.1270 - accuracy: 0.3840 - val_loss: 0.1943 - val_accuracy: 0.3597
Epoch 24/30
45/45 - 4s - loss: 0.1214 - accuracy: 0.4085 - val_loss: 0.2107 - val_accuracy: 0.3588
Epoch 25/30
45/45 - 4s - loss: 0.1256 - accuracy: 0.3887 - val_loss: 0.1931 - val_accuracy: 0.3986
Epoch 26/30
45/45 - 4s - loss: 0.1230 - accuracy: 0.3948 - val_loss: 0.2007 - val_accuracy: 0.3977
Epoch 27/30
45/45 - 4s - loss: 0.1217 - accuracy: 0.3915 - val_loss: 0.2010 - val_accuracy: 0.4597
Epoch 28/30
45/45 - 4s - loss: 0.1217 - accuracy: 0.3913 - val_loss: 0.1952 - val_accuracy: 0.3923
Epoch 29/3

Epoch 3/30
45/45 - 5s - loss: 0.1633 - accuracy: 0.2884 - val_loss: 0.1694 - val_accuracy: 0.3166
Epoch 4/30
45/45 - 4s - loss: 0.1615 - accuracy: 0.2980 - val_loss: 0.1689 - val_accuracy: 0.2522
Epoch 5/30
45/45 - 4s - loss: 0.1585 - accuracy: 0.3072 - val_loss: 0.1729 - val_accuracy: 0.2558
Epoch 6/30
45/45 - 4s - loss: 0.1622 - accuracy: 0.2996 - val_loss: 0.1662 - val_accuracy: 0.2513
Epoch 7/30
45/45 - 4s - loss: 0.1590 - accuracy: 0.2938 - val_loss: 0.1660 - val_accuracy: 0.2597
Epoch 8/30
45/45 - 4s - loss: 0.1576 - accuracy: 0.2937 - val_loss: 0.1678 - val_accuracy: 0.2806
Epoch 9/30
45/45 - 4s - loss: 0.1583 - accuracy: 0.3010 - val_loss: 0.1747 - val_accuracy: 0.2917
Epoch 10/30
45/45 - 4s - loss: 0.1538 - accuracy: 0.3049 - val_loss: 0.1576 - val_accuracy: 0.2920
Epoch 11/30
45/45 - 4s - loss: 0.1503 - accuracy: 0.3115 - val_loss: 0.1587 - val_accuracy: 0.2830
Epoch 12/30
45/45 - 4s - loss: 0.1500 - accuracy: 0.3043 - val_loss: 0.1724 - val_accuracy: 0.3154
Epoch 13/30
45/45

45/45 - 7s - loss: 0.1553 - accuracy: 0.3081 - val_loss: 0.1447 - val_accuracy: 0.2860
Epoch 2/30
45/45 - 5s - loss: 0.1426 - accuracy: 0.3225 - val_loss: 0.1421 - val_accuracy: 0.3495
Epoch 3/30
45/45 - 5s - loss: 0.1387 - accuracy: 0.3195 - val_loss: 0.1417 - val_accuracy: 0.3447
Epoch 4/30
45/45 - 5s - loss: 0.1403 - accuracy: 0.3268 - val_loss: 0.1433 - val_accuracy: 0.3360
Epoch 5/30
45/45 - 5s - loss: 0.1373 - accuracy: 0.3062 - val_loss: 0.1424 - val_accuracy: 0.3435
Epoch 6/30
45/45 - 5s - loss: 0.1378 - accuracy: 0.3223 - val_loss: 0.1423 - val_accuracy: 0.3372
Epoch 7/30
45/45 - 5s - loss: 0.1343 - accuracy: 0.3220 - val_loss: 0.1379 - val_accuracy: 0.3417
Epoch 8/30
45/45 - 5s - loss: 0.1351 - accuracy: 0.3264 - val_loss: 0.1394 - val_accuracy: 0.2986
Epoch 9/30
45/45 - 5s - loss: 0.1347 - accuracy: 0.3197 - val_loss: 0.1402 - val_accuracy: 0.3235
Epoch 10/30
45/45 - 5s - loss: 0.1336 - accuracy: 0.3245 - val_loss: 0.1337 - val_accuracy: 0.3516
Epoch 11/30
45/45 - 5s - loss:

Dans la préparation des données, (mode fenêtré), au lieu de mettre des -1 (ou des 14 en fait) à la fin de la séquence on peut mettre les vraies bases et structures

In [141]:
i=0
X_to_predict = preprocess_inputs(test_df)
X_predic=prepare_window_prediction(i,68,X_to_predict)
print(np.shape(X_predic))

seq=0

Y_predicted = np.empty(shape=(len(X_to_predict), 68, 5))

base_predict=model.predict(X_predic[seq])
Y_predicted[seq, i, :]=np.mean(base_predict[0], axis=0)



(3634, 21, 3)
[[[0.41674286 0.4671347  1.4985886  0.358269   0.4203465 ]
  [0.5142655  0.5757119  1.7937676  0.42679492 0.52134496]
  [0.57527834 0.63552916 1.9683814  0.46916822 0.5774817 ]
  [0.61128104 0.66487485 2.0638163  0.49268258 0.6061621 ]
  [0.62924284 0.6739719  2.1037922  0.5021365  0.61695665]
  [0.6331947  0.6684167  2.1020367  0.5005853  0.6151986 ]
  [0.62550944 0.65136707 2.0671666  0.4901074  0.6039386 ]
  [0.6079748  0.6251675  2.0066411  0.47261629 0.5852851 ]
  [0.58291554 0.5928929  1.9309244  0.45083216 0.561608  ]
  [0.5546226  0.5601406  1.8583722  0.42946038 0.536874  ]
  [0.53285545 0.54467964 1.8322926  0.4198663  0.52361745]
  [0.532381   0.557995   1.874204   0.42782936 0.533457  ]
  [0.542587   0.5824399  1.9385228  0.4423809  0.5521658 ]
  [0.5614021  0.61198395 2.0147612  0.46297425 0.57389116]
  [0.5848469  0.64236414 2.0905437  0.4865484  0.5957621 ]
  [0.6097378  0.66968197 2.154523   0.5096239  0.61462426]
  [0.6334111  0.68932366 2.1914537  0.5274

# Prédiction

In [ ]:
Y_predicted